In [36]:
# Ref. https://github.com/adeshpande3/LSTM-Sentiment-Analysis/blob/master/Pre-Trained%20LSTM.ipynb

In [33]:
# import os
# os.chdir('LSTM-Sentiment-Analysis/')

In [38]:
!ls

checkpoint_convert.py  models		  Pre-Trained LSTM.ipynb
Dockerfile	       models.tar.gz	  README.md
idsMatrix.npy	       negativeReviews	  training_data.tar.gz
Images		       Oriole LSTM.ipynb  wordsList.npy
LICENSE		       positiveReviews	  wordVectors.npy


In [39]:
# First, we declare our hyperparameters

In [40]:
numDimensions = 300 # like numbers of features
maxSeqLength = 250 # length of the sentence
batchSize = 24 # quantity of "data" for each iteration
lstmUnits = 64 # like number of "neurons"
numClasses = 2 # positive or negative 
iterations = 1000

In [41]:
# load our data structures

In [42]:
import numpy as np

In [43]:
wordsList = np.load('wordsList.npy').tolist()

In [44]:
wordsList = [word.decode('UTF-8') for word in wordsList] #encoding words as UTF-8

In [45]:
wordVectors = np.load('wordVectors.npy')

In [46]:
import tensorflow as tf

In [47]:
tf.reset_default_graph() 

In [48]:
labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [49]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]), dtype=tf.float32) #we'll predict here + -
data = tf.nn.embedding_lookup(wordVectors, input_data) 

In [50]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)

In [51]:
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [52]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [53]:
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [54]:
# we have to load the model and initiate session in our graph computation

In [55]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models/'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm-90000.ckpt


/home/ivan/Environments/NLP/lib/python3.5/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [56]:
# before input our own text, define some functions

In [57]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

In [58]:
def cleanSenteces(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, '', string.lower())

In [60]:
def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize, maxSeqLength], dtype='int32')
    cleanedSentence = cleanSenteces(sentence)
    split = cleanedSentence.split()
    for indexCounter, word in enumerate(split):
        try:
            sentenceMatrix[0, indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0, indexCounter] = 399999
    return sentenceMatrix

In [61]:
inputText = 'That movie was terrible'
inputMatrix = getSentenceMatrix(inputText)

In [62]:
predictSentiment = sess.run(prediction, {input_data: inputMatrix})[0]

In [63]:
if (predictSentiment[0] > predictSentiment[1]):
    print('positive sentiment')
else:
    print('negative sentiment')

negative sentiment
